**Análise estatística dos citoesqueletos extraídos do Fiji/ImageJ**

In [1]:
# Math, image processing and other useful libraries
from __future__ import print_function, unicode_literals, absolute_import, division
import os
import pandas as pd
import numpy as np
import cv2
from collections import OrderedDict
import copy
import math
import pickle
from matplotlib.ticker import MaxNLocator
from itertools import combinations

# Image processing
from skimage.measure import regionprops
from skimage.filters import meijering, sato, frangi, hessian, threshold_otsu
from skimage.morphology import extrema, skeletonize
from skimage.transform import probabilistic_hough_line
from skimage.draw import disk, circle_perimeter
from scipy.ndimage import gaussian_filter, grey_closing
from scipy.spatial import distance_matrix
from skimage import data, restoration, util
from roipoly import RoiPoly
from matplotlib_scalebar.scalebar import ScaleBar
from biosppy.signals import tools
from biosppy.stats import pearson_correlation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting
import matplotlib.pyplot as plt
import matplotlib.cm as pltc
import matplotlib.colors as colors
import seaborn as sns

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Nuclei Segmentation
from auxiliary_functions_segmentation import segment_patches

# Feature Extraction (.py files by Teresa Parreira)
from CytoSkeletonPropsMorph import CytoSkeletonPropsMorph
from CytoSkeletonRegionPropsInt import RegionPropsInt
from FreqAnalysis import FreqAnalysis
from GLCM import GLCM

# 
from Functions import img_getTexture, label_image, FeaturesFromCentroid, cv2toski,pylsdtoski,init_import,getCentroids,polar_to_cartesian, truncate_colormap, plot_hist, plot_pie, remove_not1D, analyze_cell, quantitative_analysis,hist_bin,hist_lim 
from fractal_dimension import fractal_dimension
from fractal_analysis_fxns import boxcount,boxcount_grayscale,fractal_dimension,fractal_dimension_grayscale,fractal_dimension_grayscale_DBC

C:\Users\diogo\AppData\Roaming\Python\Python37\site-packages\nose\importer.py:12: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import find_module, load_module, acquire_lock, release_lock


# Import data

In [2]:
folder = os.getcwd() + "//Skeletons//"

folds    = [len(x) < 3 for x in os.listdir(folder)]
img_idx  = os.listdir(folder)[:-1]

FijiDF = pd.DataFrame(columns = ['Cell patch','Image','Cell label','Cell desc.','Cell class','Branch Information','Results'])
     
global row
for i in img_idx:
    d = folder + str(i) + "//"
    
    global cell
    for cell in os.listdir(d):
        d_cell = d + str(cell)
        
        # Import Branch Information
        BI = pd.read_csv(d_cell + str("//Branch Information.csv"))
        
        # Import Results
        R  = pd.read_csv(d_cell + str("//Results.csv"))
        
        # Import images
        tagged     = cv2.imread(d_cell + str("//Tagged skeleton.tif"),cv2.IMREAD_COLOR)
        cell_patch = cv2.imread(d_cell + str("//") + str(i) + str("-labeled-skeletons.tif"),-1)
        # TO DO
#         from PIL import Image
#         for image in os.listdir(d_cell):
#             image_dir = d_cell + str("//") + str(image)
#             if image.endswith(".tif"):
#                 im = plt.imread(image_dir)
#                 plt.imsave(image + str(".png"),"png")

        new = pd.Series([cell_patch] + [i] + [label_image(int(i))] + [cell] + [cell.split('_')[0]] + [BI] + [R],index=FijiDF.columns)
        FijiDF = FijiDF.append(new,ignore_index=True)
        
# Preprocessing
for index,row in FijiDF.iterrows():
    # Branch Information
    bi = row['Branch Information']
    row['Branch Information'] = bi[bi.columns[:-2]]
    
    # Results
    r  = row['Results']
    row['Results'] = r[r.columns[1:-1]]
    
# Statistics
global fts,res,temp,StatisticsDF
StatisticsDF = pd.DataFrame()
for index,row in FijiDF.iterrows():
    # Add Branch Information Statistics
    bi = row['Branch Information']
    fts = tools.signal_stats(bi[bi.columns[0]])._names
    res = pd.DataFrame([row[:5].values],columns = FijiDF.columns[:5])
    global ft_bi
    for ft_bi in bi.columns: 
        temp = np.array(list(tools.signal_stats(bi[ft_bi])))
        res = pd.concat([res, pd.DataFrame([temp],columns = [ft_bi+str(" ")+i for i in fts])],axis=1)
    
    # Add Results Statistics
    r = row['Results']
    global ft_r
    for ft_r in r.columns:
        temp = np.array(list(tools.signal_stats(r[ft_r])))
        res = pd.concat([res, pd.DataFrame([temp],columns = [ft_r+str(" ")+i for i in fts])],axis=1)
    
    StatisticsDF = pd.concat([StatisticsDF, res],axis=0)

In [3]:
FijiDF

,Cell patch,Image,Cell label,Cell desc.,Cell class,Branch Information,Results
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",11,WT,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",40,Del38_46,complex_left,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",40,Del38_46,complex_right,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",59,Dup41_46,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",63,Dup41_46,complex_paralelogram,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
5,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",63,Dup41_46,complex_triangular,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
6,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",66,Dup41_46,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
7,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,complex_bot,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
8,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,complex_top,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
9,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...


In [5]:
FijiDF.loc[0]['Branch Information']

,Skeleton ID,Branch length,V1 x,V1 y,V1 z,V2 x,V2 y,V2 z,Euclidean distance,running average length
0,1,30.556,23,547,0,48,535,0,27.731,28.723
1,2,18.728,59,566,0,66,551,0,16.553,17.203
2,2,16.314,72,544,0,82,533,0,14.866,14.004
3,2,13.721,54,548,0,59,537,0,12.083,10.595
4,2,11.657,54,575,0,59,566,0,10.296,10.675
...,...,...,...,...,...,...,...,...,...,...
399,97,9.071,237,81,0,230,76,0,8.602,6.619
400,97,5.657,226,74,0,230,76,0,4.472,5.361
401,98,1.000,230,142,0,230,143,0,1.000,1.000
402,99,1.000,241,101,0,241,102,0,1.000,1.000


In [4]:
#StatisticsDF = StatisticsDF.sort_values(by=['Cell class'])
StatisticsDF

,Cell patch,Image,Cell label,Cell desc.,Cell class,Skeleton ID mean,Skeleton ID median,Skeleton ID min,Skeleton ID max,Skeleton ID max_amp,...,spy mean,spy median,spy min,spy max,spy max_amp,spy var,spy std_dev,spy abs_dev,spy kurtosis,spy skewness
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",11,WT,elongated,elongated,37.698020,41.0,1.0,100.0,62.301980,...,308.730000,326.5,28.0,547.0,280.730000,17110.643535,130.807659,111.270000,-0.982066,-0.150099
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",40,Del38_46,complex_left,complex,82.020468,83.0,1.0,175.0,92.979532,...,148.737143,152.0,5.0,311.0,162.262857,5026.114417,70.895094,60.920000,-1.036571,-0.009277
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",40,Del38_46,complex_right,complex,72.502358,68.5,1.0,156.0,83.497642,...,230.743590,241.5,8.0,423.0,222.743590,9635.559636,98.160886,77.820513,-0.448458,-0.271026
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",59,Dup41_46,elongated,elongated,21.618474,28.0,1.0,55.0,33.381526,...,176.890909,189.0,32.0,322.0,145.109091,5379.284175,73.343604,63.018182,-1.015327,-0.051823
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",63,Dup41_46,complex_paralelogram,complex,42.386423,39.0,1.0,106.0,63.613577,...,142.594340,137.0,27.0,278.0,135.405660,2384.624349,48.832616,37.858491,0.193250,0.368087
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",63,Dup41_46,complex_triangular,complex,65.101896,63.5,1.0,137.0,71.898104,...,191.430657,201.0,21.0,339.0,170.430657,5923.496994,76.964258,64.445255,-0.826771,-0.226742
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",66,Dup41_46,elongated,elongated,20.857923,25.0,1.0,52.0,31.142077,...,172.750000,178.0,17.0,321.0,155.750000,4817.250000,69.406412,51.865385,0.308588,-0.048538
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,complex_bot,complex,22.355596,16.0,1.0,96.0,73.644404,...,232.822917,232.0,53.0,350.0,179.822917,4361.178838,66.039222,53.760417,-0.472283,-0.097396
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,complex_top,complex,31.447266,25.0,1.0,94.0,62.552734,...,154.819149,153.0,14.0,323.0,168.180851,4453.762640,66.736517,55.031915,-0.537762,0.063933
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",9,WT,elongated,elongated,17.776786,20.5,2.0,52.0,34.223214,...,409.269231,468.0,15.0,814.0,404.730769,43149.377074,207.724281,169.884615,-0.666135,-0.060656


In [1]:
def cellvscell(DF):
    DF = DF.sort_values(by=['Cell class'])
    
    global labels, colors, markers
    labels  = np.unique(DF['Cell class'])
    colors  = ["#3498DB","#E74C3C","#95A5A6"]
    markers = ["o","v","^"]
    
    # LSF DATAFRAME
#     global LSF,StatsDF,c
#     LSF = CellSubsetDF[CellSubsetDF.columns[2:8]] #por 9
#     fts = tools.signal_stats(LSF.loc[LSF.index[0]]['Angles'])._names
#     res = pd.DataFrame()
#     for ft in LSF.columns: 
#         temp = np.array([list(tools.signal_stats(cell)) for cell in LSF[ft]])
#         res  = pd.concat([res, pd.DataFrame(temp,columns = [ft+str(" ")+i for i in fts])],axis=1)

#     StatsDF = pd.concat([res, pd.DataFrame(np.array(CellSubsetDF[CellSubsetDF.columns[8]]),columns=[ResultsDF.columns[8]])],axis=1)
    
    # Labels
    c    = [colors[list(labels).index(x)] for x in DF['Cell class']]
    m    = [markers[list(labels).index(y)] for y in DF['Cell class']]

#     # Bar plot
#     i = 0
#     for mainft in LSF.columns:
#         pl = 1
#         fig, ax = plt.subplots(figsize=(15,5))
#         plt.suptitle(mainft)
#         for _ in range(10):
#             plt.subplot(2,5,pl)
#             plt.bar(range(len(CellSubsetDF.index)),StatsDF[StatsDF.columns[i]],color=c,label=labels)
#             plt.ylabel(StatsDF.columns[i],fontsize=8)
#             #plt.legend()
#             plt.xticks(range(len(CellSubsetDF.index)),CellSubsetDF.index,fontsize=7,rotation=45)
#             plt.tight_layout()
#             pl += 1
#             i  +=1
        
#         plt.legend()
#         plt.show()
        
    # 1D feats
    global feat
    for feat in DF.columns[5:]:
        fig, ax = plt.subplots(figsize=(15,8))
        cmap = pltc.Reds
        new_cmap = truncate_colormap(cmap, 0.3, 1, 300)
        #plt.bar(range(len(CellSubsetDF.index)),CellSubsetDF[CellSubsetDF.columns[8]],color=[new_cmap(plt.Normalize(0, max(CellSubsetDF[CellSubsetDF.columns[8]]))(c)) for c in CellSubsetDF[CellSubsetDF.columns[8]]],alpha=0.7)
        global ind
        ind = 0
        global aux
        for l in labels:
            aux = DF[DF['Cell class'] == l]
            plt.bar(range(ind,ind + len(aux)),aux[feat],label=l)
            ind = ind + len(aux)
        #plt.bar(range(len(CellSubsetDF.index)),CellSubsetDF[feat],color=c,label=labels)
        plt.xticks(range(len(DF.index)), StatisticsDF['Image'] +str(",") + StatisticsDF['Cell label'], fontsize=12,rotation=45)
        #plt.title(feat)
        plt.ylabel(feat)
        plt.tight_layout()
        plt.legend()
        #plt.savefig(".//CellVSCellAnalysis//features//" + str(index) + str("_") + str(feat) + ".png",format='png',transparent=True,bbox_inches='tight')
        plt.show()

    
    plt.show()

cellvscell(StatsDF)

NameError: name 'StatsDF' is not defined

In [127]:
def boxplot_Fiji(DF):
    
    fig, ax = plt.subplots(figsize=(8, 5))
    #plt.title('Boxplot - ' + feat,fontsize=20)

    boxhelp = []
    for label in labels:
        temp = box[label]
        boxhelp += [temp[~np.isnan(temp)]]
    bplot = ax.boxplot(np.transpose(boxhelp),patch_artist=True,notch=True,labels=labels)

    for patch,color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    plt.ylabel(feat,fontsize=16)
    ax.yaxis.grid(True)
    #plt.savefig(".//ResultsGenAnalysis//Hough26Mar//" + str(feat) + "_Boxplot.png",format='png',transparent=True,bbox_inches='tight')
    plt.show()

In [126]:
row['Branch Information']

,Skeleton ID,Branch length,V1 x,V1 y,V1 z,V2 x,V2 y,V2 z,Euclidean distance,running average length
0,1,46.213,75,136,0,44,116,0,36.892,42.884
1,1,13.899,59,117,0,48,110,0,13.038,12.927
2,1,12.893,58,109,0,48,105,0,10.770,12.054
3,1,9.071,42,105,0,48,105,0,6.000,6.955
4,1,8.414,38,107,0,41,113,0,6.708,6.763
...,...,...,...,...,...,...,...,...,...,...
150,45,2.000,133,91,0,135,91,0,2.000,2.000
151,46,1.000,134,122,0,134,123,0,1.000,1.000
152,47,6.828,135,129,0,140,126,0,5.831,6.122
153,48,9.243,139,114,0,144,120,0,7.810,8.533


In [17]:
FijiDF

,Image,Cell desc.,Cell class,Branch Information,Results
0,11,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
1,40,complex_left,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
2,40,complex_right,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
3,59,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
4,63,complex_paralelogram,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
5,63,complex_triangular,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxe...
6,66,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
7,9,complex_bot,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
8,9,complex_top,complex,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
9,9,elongated,elongated,Skeleton ID Branch length V1 x V1 y V...,# Branches # Junctions # End-point voxel...
